In [19]:
#create dataset
%%bigquery
CREATE SCHEMA IF NOT EXISTS emergency_calls_dataset
OPTIONS(
 location="us");

#create Table
CREATE OR REPLACE EXTERNAL TABLE `qwiklabs-gcp-02-e82d432d54e9.emergency_calls_dataset.emergency_calls`
OPTIONS (
  format = 'CSV',
  uris = ['gs://labs.roitraining.com/data-to-ai-workshop/emergency_calls_response_times.csv'],
  skip_leading_rows = 1,
  allow_jagged_rows = false,
  allow_quoted_newlines = false,
  field_delimiter = ',',
  max_bad_records = 10
)

Query is running:   0%|          |

""


In [20]:
%%bigquery
CREATE OR REPLACE TABLE `qwiklabs-gcp-02-e82d432d54e9.emergency_calls_dataset.emergency_calls_encoded` AS(
  SELECT
  * EXCEPT (
    call_type, weather_condition, day_of_week, traffic_level
  ),
  -- hot encode
  -- call_type
  CASE WHEN call_type = 'Police' THEN 1 ELSE 0 END AS call_type_police,
  CASE WHEN call_type = 'Medical' THEN 1 ELSE 0 END AS call_type_medical,
  CASE WHEN call_type = 'Fire' THEN 1 ELSE 0 END AS call_type_fire,
  CASE WHEN call_type = 'Rescue' THEN 1 ELSE 0 END AS call_type_rescue,
  --weather_condition
  CASE WHEN weather_condition = 'Rainy' THEN 1 ELSE 0 END AS weather_condition_rainy,
  CASE WHEN weather_condition = 'Snowy' THEN 1 ELSE 0 END AS weather_condition_snowy,
  CASE WHEN weather_condition = 'Sunny' THEN 1 ELSE 0 END AS weather_condition_sunny,
  CASE WHEN weather_condition = 'Windy' THEN 1 ELSE 0 END AS weather_condition_windy,

  --day of the week
    CASE WHEN day_of_week = 'Sunday' THEN 1 ELSE 0 END AS day_of_week_sunday,
    CASE WHEN day_of_week = 'Monday' THEN 1 ELSE 0 END AS day_of_week_monday,
    CASE WHEN day_of_week = 'Tuesday' THEN 1 ELSE 0 END AS day_of_week_tuesday,
    CASE WHEN day_of_week = 'Wednesday' THEN 1 ELSE 0 END AS day_of_week_wednesday,
    CASE WHEN day_of_week = 'Thursday' THEN 1 ELSE 0 END AS day_of_week_thursday,
    CASE WHEN day_of_week = 'Friday' THEN 1 ELSE 0 END AS day_of_week_friday,
    CASE WHEN day_of_week = 'Saturday' THEN 1 ELSE 0 END AS day_of_week_saturday,
  -- traffic_level
    CASE WHEN traffic_level = 'High' THEN 1 ELSE 0 END AS traffic_level_high,
    CASE WHEN traffic_level = 'Medium' THEN 1 ELSE 0 END AS traffic_level_medium,
    CASE WHEN traffic_level = 'Low' THEN 1 ELSE 0 END AS traffic_level_low,

   FROM `qwiklabs-gcp-02-e82d432d54e9.emergency_calls_dataset.emergency_calls`

);



Query is running:   0%|          |

""


In [21]:
#create ML Model and Train

%%bigquery create_model
CREATE or REPLACE MODEL `qwiklabs-gcp-02-e82d432d54e9.emergency_calls_dataset.response_time_model`
OPTIONS(
    model_type='LINEAR_REG',
    input_label_cols=['response_time']) AS
SELECT * FROM  `qwiklabs-gcp-02-e82d432d54e9.emergency_calls_dataset.emergency_calls_encoded`
WHERE response_time IS NOT NULL AND call_timestamp BETWEEN '2023-01-01' AND '2023-11-30'; -- Ensure target variable is not null for training

Query is running:   0%|          |

In [22]:
#evalutate the ML Model
%%bigquery
SELECT * FROM ML.EVALUATE(
  MODEL `qwiklabs-gcp-02-e82d432d54e9.emergency_calls_dataset.response_time_model`
)


Query is running:   0%|          |

Downloading:   0%|          |

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,1.745031,4.783751,0.014862,1.469867,0.82902,0.829173


In [27]:
# Predict
%%bigquery
SELECT * FROM ML.PREDICT (
  MODEL `qwiklabs-gcp-02-e82d432d54e9.emergency_calls_dataset.response_time_model`, (
    SELECT * FROM `qwiklabs-gcp-02-e82d432d54e9.emergency_calls_dataset.emergency_calls_encoded`
    WHERE response_time IS NOT NULL AND call_timestamp BETWEEN '2023-12-01' AND '2023-12-31'

  )

)


Query is running:   0%|          |

Downloading:   0%|          |

,predicted_response_time,call_id,call_timestamp,location,time_of_day,distance_to_station,units_available,response_time,call_type_police,call_type_medical,...,day_of_week_sunday,day_of_week_monday,day_of_week_tuesday,day_of_week_wednesday,day_of_week_thursday,day_of_week_friday,day_of_week_saturday,traffic_level_high,traffic_level_medium,traffic_level_low
0,-22810.114594,29367,2023-12-01 00:02:58+00:00,Oakmont,0,7.82,9,14.15,0,0,...,0,0,0,0,0,1,0,0,0,1
1,-22799.873081,25304,2023-12-01 00:04:17+00:00,Downtown,0,23.42,9,19.89,0,0,...,0,0,0,0,0,1,0,0,0,1
2,-22799.110105,38905,2023-12-01 00:07:11+00:00,Riverside,0,27.55,2,28.93,0,0,...,0,0,0,0,0,1,0,1,0,0
3,-22807.897843,44026,2023-12-01 00:09:39+00:00,Downtown,0,9.34,3,15.28,0,0,...,0,0,0,0,0,1,0,0,0,1
4,-22796.004905,46123,2023-12-01 00:24:04+00:00,Greenfield,0,29.25,2,25.73,1,0,...,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4151,7286.132474,20813,2023-12-30 21:28:37+00:00,Maplewood,21,25.37,6,21.76,0,0,...,0,0,0,0,0,0,1,0,1,0
4152,7287.370510,44525,2023-12-30 21:40:21+00:00,Brookfield,21,23.22,10,19.12,0,0,...,0,0,0,0,0,0,1,0,1,0
4153,7280.768710,13740,2023-12-30 23:21:57+00:00,Uptown,23,23.20,14,16.88,1,0,...,0,0,0,0,0,0,1,0,1,0
4154,7277.409514,7395,2023-12-30 23:41:59+00:00,Greenfield,23,5.70,2,14.79,0,0,...,0,0,0,0,0,0,1,0,1,0


In [26]:
prediction

,predicted_response_time,call_id,call_timestamp,location,time_of_day,distance_to_station,units_available,response_time,call_type_police,call_type_medical,...,day_of_week_sunday,day_of_week_monday,day_of_week_tuesday,day_of_week_wednesday,day_of_week_thursday,day_of_week_friday,day_of_week_saturday,traffic_level_high,traffic_level_medium,traffic_level_low
0,-22810.114594,29367,2023-12-01 00:02:58+00:00,Oakmont,0,7.82,9,14.15,0,0,...,0,0,0,0,0,1,0,0,0,1
1,-22799.873081,25304,2023-12-01 00:04:17+00:00,Downtown,0,23.42,9,19.89,0,0,...,0,0,0,0,0,1,0,0,0,1
2,-22799.110105,38905,2023-12-01 00:07:11+00:00,Riverside,0,27.55,2,28.93,0,0,...,0,0,0,0,0,1,0,1,0,0
3,-22807.897843,44026,2023-12-01 00:09:39+00:00,Downtown,0,9.34,3,15.28,0,0,...,0,0,0,0,0,1,0,0,0,1
4,-22796.004905,46123,2023-12-01 00:24:04+00:00,Greenfield,0,29.25,2,25.73,1,0,...,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4151,7286.132474,20813,2023-12-30 21:28:37+00:00,Maplewood,21,25.37,6,21.76,0,0,...,0,0,0,0,0,0,1,0,1,0
4152,7287.370510,44525,2023-12-30 21:40:21+00:00,Brookfield,21,23.22,10,19.12,0,0,...,0,0,0,0,0,0,1,0,1,0
4153,7280.768710,13740,2023-12-30 23:21:57+00:00,Uptown,23,23.20,14,16.88,1,0,...,0,0,0,0,0,0,1,0,1,0
4154,7277.409514,7395,2023-12-30 23:41:59+00:00,Greenfield,23,5.70,2,14.79,0,0,...,0,0,0,0,0,0,1,0,1,0
